In [7]:
import joblib
import cv2
import numpy as np
from skimage.feature import hog
from keras.models import load_model

In [8]:
IMG_SIZE = 224

In [9]:
# ============================
# LOAD ALL MODELS
# ============================
print("[INFO] Loading models...")

knn = joblib.load("knn_model.pkl")
rf = joblib.load("random_forest_model.pkl")
scaler = joblib.load("scaler.pkl")
encoder = joblib.load("encoder.pkl")

cnn_model = load_model("inception_model.h5")

[INFO] Loading models...


In [10]:
# ============================
# FEATURE EXTRACTION FUNCTION
# ============================
def extract_features_for_ml(img):
    img_resized = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    gray = cv2.cvtColor(img_resized, cv2.COLOR_BGR2GRAY)

    hog_feature = hog(gray, orientations=9, pixels_per_cell=(16,16),
                      cells_per_block=(2,2), block_norm="L2-Hys")

    hist = cv2.calcHist([img_resized], [0,1,2], None, [8,8,8],
                        [0,256,0,256,0,256]).flatten()

    return np.hstack([hog_feature, hist])

In [31]:
# ============================
# MAIN TEST FUNCTION
# ============================
def test_image(img_path):
    print(f"\n[INFO] Testing image: {img_path}")

    img = cv2.imread(img_path)
    if img is None:
        print("[ERROR] Gambar gak kebaca.")
        return

    # ----- ML FEATURES -----
    feat = extract_features_for_ml(img).reshape(1, -1)
    feat_scaled = scaler.transform(feat)

    # ----- KNN -----
    knn_probs = knn.predict_proba(feat_scaled)[0]
    knn_idx = np.argmax(knn_probs)
    pred_knn = encoder.classes_[knn_idx]
    conf_knn = knn_probs[knn_idx] * 100

    # ----- Random Forest -----
    rf_probs = rf.predict_proba(feat_scaled)[0]
    rf_idx = np.argmax(rf_probs)
    pred_rf = encoder.classes_[rf_idx]
    conf_rf = rf_probs[rf_idx] * 100

    # ----- CNN -----
    img_cnn = cv2.resize(img, (IMG_SIZE, IMG_SIZE)) / 255.0
    img_cnn = np.expand_dims(img_cnn, axis=0)

    probs = cnn_model.predict(img_cnn)[0]
    pred_cnn_idx = np.argmax(probs)
    pred_cnn = encoder.classes_[pred_cnn_idx]
    conf_cnn = probs[pred_cnn_idx] * 100

    # ========================
    # PRINT RESULTS
    # ========================
    print("\n======= RESULT =======")
    print(f"KNN Prediction                  : {pred_knn} ({conf_knn:.2f}%)")
    print(f"Random Forest Prediction        : {pred_rf} ({conf_rf:.2f}%)")
    print(f"CNN + InceptionV3 Prediction    : {pred_cnn} ({conf_cnn:.2f}%)")
    print("=======================")

In [32]:
# ============================
# RUN TEST
# ============================
if __name__ == "__main__":
    # Medium
    print("Test 1 (Medium)")
    print("================================================")
    test_image("datasets/test/Medium/medium (42).png")

    # Dark
    print("\nTest 2 (Dark)")
    print("================================================")
    test_image("datasets/test/Dark/dark (42).png")

    # Green
    print("\nTest 3 (Green)")
    print("================================================")
    test_image("datasets/test/Green/green (42).png")

    # Light
    print("\nTest 4 (Light)")
    print("================================================")
    test_image("datasets/test/Light/light (42).png")

Test 1 (Medium)

[INFO] Testing image: datasets/test/Medium/medium (42).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step

======= RESULT =======
KNN Prediction                  : Dark (66.67%)
Random Forest Prediction        : Medium (91.25%)
CNN + InceptionV3 Prediction    : Medium (54.28%)

Test 2 (Dark)

[INFO] Testing image: datasets/test/Dark/dark (42).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step

======= RESULT =======
KNN Prediction                  : Dark (66.67%)
Random Forest Prediction        : Dark (97.50%)
CNN + InceptionV3 Prediction    : Dark (99.83%)

Test 3 (Green)

[INFO] Testing image: datasets/test/Green/green (42).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step



======= RESULT =======
KNN Prediction                  : Green (100.00%)
Random Forest Prediction        : Green (92.50%)
CNN + InceptionV3 Prediction    : Green (63.05%)

Test 4 (Light)

[INFO] Testing image: datasets/test/Light/light (42).png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step

======= RESULT =======
KNN Prediction                  : Light (100.00%)
Random Forest Prediction        : Light (86.25%)
CNN + InceptionV3 Prediction    : Light (93.32%)
